In [3]:
# %env SM_FRAMEWORK=tf.keras
import zipfile, os, numpy as np, pickle, yaml, gc, tensorflow as tf
import segmentation_models as sm
import tensorflow_addons as tfa
sys.path.append("..")
from keras import backend as K
K.clear_session()
from sklearn.metrics import jaccard_score
from segmentation_models import Unet
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"
class_type = 0 # 0=NL, 1=AP

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [4]:
np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]).astype(np.int8)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int8)

In [4]:
# path test
top_layer_path_train = '../my_data/datasets/all_seg_data_NL_zm_2/'
top_layer_path = '../my_data/datasets/test_data_NL/'
# S1 [img, msk]
S1_img_stack = ['T3_image_arr_384_valid.npy', 'T3_masks_arr_384_valid.npy', 'T3_image_arr_384_train.npy', 'T3_masks_arr_384_train.npy']
# save path
# save_path = f'results_log/S1/all_S1-test_results_{datatype}'

# S1 model weight path
S1_weights=['../my_data/my_weights/S1_DenseNet.hdf5','../my_data/my_weights/S1_ResNet.hdf5','../my_data/my_weights/S1_VGGNet.hdf5']
# S1_weights=['./checkpoints/S1-segment/best-valid-auc_2DDenseUnet-12.16-original_3T_NL-data_dice-loss.hdf5', './checkpoints/S1-segment/best-valid-auc_2DResNet50-12.16-original_3T_NL-data_dice-loss.hdf5', config["S1_vgg"]]
S1_backbone = ['densenet121', 'resnet50', 'vgg16']


In [5]:
import pandas as pd
compare_csv = pd.read_csv('../paper_reviewer/Only Test data.csv')
# print(compare_csv.columns)
fiter_csv = np.array(compare_csv[['Test MRI', 'Mean Image Intensity', 'Mask Pixel Sum']])
mri_value = np.array([i for i in fiter_csv if 'is' in i[0]])
# print(mri_value)
mri_compare_value = np.array([i for i in fiter_csv if ('is' not in i[0] and 'overall' in i[0])])
# print(mri_compare_value)
mri_value_filter_intensity = np.array([1 if i[1]>mri_compare_value[3][1] else 0 for i in mri_value ])
print(mri_value_filter_intensity)
mri_value_filter_pixel = np.array([1 if i[2]>mri_compare_value[3][2] else 0 for i in mri_value ])
print(mri_value_filter_pixel)

[0 0 0 1 0 0 0 1 1 0 1 1 1 1 1 1 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0
 1 0 0 0 0 1 0 0]
[0 1 0 0 0 0 1 0 1 1 0 0 0 1 0 1 0 1 1 0 0 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 0
 1 0 0 0 0 0 0 0]


In [6]:
def S1_dataloader(valid_data='Mix', tune_type='test', type_data=None):
    if tune_type=='test':
        img_layer_path = top_layer_path
        if valid_data == '3.0T':
            # loading valida data 3.0T + 1.5T: image / masks
            X_valid = np.load(img_layer_path +'/'+ S1_img_stack[0])
            y_valid = np.load(img_layer_path +'/'+ S1_img_stack[1])
    elif tune_type=='train':
        img_layer_path = top_layer_path_train
        X_valid = np.load(img_layer_path +'/'+ S1_img_stack[2])
        y_valid = np.load(img_layer_path +'/'+ S1_img_stack[3])
    # print(len(experiment_stack[type_data]))
    # print(np.where(experiment_stack[type_data]==1))
    
    if ">" in type_data:
        X_valid =  X_valid[np.where(experiment_stack[type_data]==1)]
        y_valid = y_valid[np.where(experiment_stack[type_data]==1)]
    else:
        # print(np.where(experiment_stack[type_data]==0))
        X_valid = X_valid[np.where(experiment_stack[type_data]==0)]
        y_valid = y_valid[np.where(experiment_stack[type_data]==0)]
    print(X_valid.shape)
    X_valid = np.reshape(X_valid, (X_valid.shape[0]*32,384,384,1))
    y_valid = np.reshape(y_valid, (y_valid.shape[0]*32,384,384,1))
    return X_valid.astype(np.float32), y_valid.astype(np.float32)

In [7]:
def S1_model_loader(weight_path, backbone, mode, type_data=None):
    S1_X_valid, S1_y_valid= S1_dataloader(valid_data='3.0T', tune_type=mode, type_data=type_data)
    print(f'S1 data shape: img {S1_X_valid.shape} msk {S1_y_valid.shape}')
    model = Unet(backbone, encoder_weights=None, input_shape=(None, None, 1))
    model.load_weights(weight_path)
    Results = model.predict(S1_X_valid, batch_size=1, verbose=1)
    return np.array(Results), np.array(S1_X_valid), np.array(S1_y_valid)

In [8]:
from sklearn.metrics import confusion_matrix
S1_thresholds = 0.5
S1_pred_stack=[]
class metric_:
    def __init__(self,y_true, y_pred):
        self.y_true_f = K.flatten(y_true)
        self.y_pred_f = K.flatten(y_pred)
        self.current = confusion_matrix(K.cast(self.y_true_f, dtype='int8'), K.cast(self.y_pred_f, dtype='int8'), labels=[0, 1])
    def dice_coef(self):
        # intersection = K.sum(self.y_true_f* self.y_pred_f )
        intersection = np.diag(self.current)
        ground_truth_set = self.current.sum(axis=1)
        predicted_set = self.current.sum(axis=0)
        # dice = (2. * intersection) / (K.sum(self.y_true_f) + K.sum(self.y_pred_f ))
        dice = (2. * intersection) / (ground_truth_set + predicted_set)
        return np.mean(dice)
        
    def iou(self):
        # ytrue, ypred is a flatten vector
        # compute mean iou
        intersection = np.diag(self.current)
        ground_truth_set = self.current.sum(axis=1)
        predicted_set = self.current.sum(axis=0)
        union = ground_truth_set + predicted_set - intersection
        IoU = intersection / union.astype(np.float32)
        return np.mean(IoU)

In [7]:
# ----S1 model test 1 - 3----
# mode_list=['test', 'train']
experiment_stack = {'Mean Intensity < Overall Mean': mri_value_filter_intensity, 'Mean Intensity > Overall Mean': mri_value_filter_intensity, 'Sum Pixel < Overall Mean': mri_value_filter_pixel , 'Sum Pixel > Overall Mean': mri_value_filter_pixel}
mode_list=['test']
for idx,d in enumerate(mode_list):
    print(f'Start model = {d}')
    for i in experiment_stack:
        print(f'----------{i}-------------↓ ')
        S1_pred, S1_X_valid, S1_y_valid = S1_model_loader(S1_weights[1], S1_backbone[1], d,type_data= i)
        metric_class = metric_(S1_y_valid,S1_pred)
        print("Dice: ", metric_class.dice_coef())
        print("IoU:  ", metric_class.iou())

Start model = test
----------Mean Intensity < Overall Mean-------------↓ 
(28, 32, 384, 384)
S1 data shape: img (896, 384, 384, 1) msk (896, 384, 384, 1)
896/896 [==============================] - 29s 33ms/step
Dice:  0.8940396846904755
IoU:   0.8250696920089773
----------Mean Intensity > Overall Mean-------------↓ 
(17, 32, 384, 384)
S1 data shape: img (544, 384, 384, 1) msk (544, 384, 384, 1)
544/544 [==============================] - 17s 31ms/step
Dice:  0.9190784403555322
IoU:   0.8606600305236347
----------Sum Pixel < Overall Mean-------------↓ 
(28, 32, 384, 384)
S1 data shape: img (896, 384, 384, 1) msk (896, 384, 384, 1)
896/896 [==============================] - 28s 31ms/step
Dice:  0.8785297871706145
IoU:   0.8045362148305577
----------Mean Pixel > Overall Mean-------------↓ 
(17, 32, 384, 384)
S1 data shape: img (544, 384, 384, 1) msk (544, 384, 384, 1)
544/544 [==============================] - 16s 29ms/step
Dice:  0.9048207818705796
IoU:   0.8399449721266359
